In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

import pandas as pd

In [5]:
train_data_file = 'data/zhengqi_train.txt'
test_data_file = 'data/zhengqi_test.txt'

train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')

train_data = train_data[train_data['V9']>-7.5]
test_data = test_data[test_data['V9']>-7.5]

from sklearn.decomposition import PCA
from sklearn import preprocessing

feature_columns = [col for col in train_data.columns if col not in ['target']]
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(train_data[feature_columns])

train_data_scaler = min_max_scaler.transform(train_data[feature_columns])
test_data_scaler = min_max_scaler.transform(test_data[feature_columns])

train_data_scaler = pd.DataFrame(train_data_scaler)
train_data_scaler.columns = feature_columns
train_data_scaler['target'] = train_data['target']

test_data_scaler = pd.DataFrame(test_data_scaler)
test_data_scaler.columns = feature_columns

pca = PCA(n_components=16)
new_train_pca_16 = pca.fit_transform(train_data_scaler.iloc[:, 0:-1])
new_test_pca_16 = pca.transform(test_data_scaler)
new_train_pca_16 = pd.DataFrame(new_train_pca_16)
new_test_pca_16 = pd.DataFrame(new_test_pca_16)

new_train_pca_16['target'] = train_data_scaler['target']
new_train_pca_16.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,target
count,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2.886000e+03,2884.000000
mean,-3.939253e-17,-1.969626e-17,2.954440e-17,-5.908879e-17,7.386099e-17,-2.708236e-17,5.908879e-17,5.047168e-17,-4.924066e-17,8.740217e-17,9.848132e-18,1.280257e-16,-1.538771e-17,-5.293371e-17,-9.848132e-18,1.335653e-16,0.127274
std,3.998976e-01,3.500240e-01,2.938631e-01,2.728023e-01,2.077128e-01,1.951842e-01,1.877104e-01,1.607670e-01,1.512707e-01,1.443772e-01,1.368790e-01,1.286191e-01,1.193301e-01,1.149757e-01,1.133507e-01,1.019258e-01,0.983462
min,-1.071795e+00,-9.429479e-01,-9.948314e-01,-7.103086e-01,-7.703985e-01,-5.340284e-01,-5.993764e-01,-5.870792e-01,-6.283013e-01,-4.902477e-01,-6.340768e-01,-5.905543e-01,-4.175720e-01,-4.309496e-01,-4.170885e-01,-3.602942e-01,-3.044000
25%,-2.804085e-01,-2.613727e-01,-2.090798e-01,-1.945196e-01,-1.315623e-01,-1.264092e-01,-1.236355e-01,-1.016389e-01,-9.663332e-02,-9.297162e-02,-8.202524e-02,-7.722117e-02,-7.140315e-02,-7.490259e-02,-7.712204e-02,-6.605996e-02,-0.348500
50%,-1.417104e-02,-1.277241e-02,2.112166e-02,-2.337395e-02,-5.122861e-03,-1.355346e-02,-1.746628e-04,-4.656764e-03,2.574516e-03,-1.475378e-03,7.289760e-03,-5.671208e-03,-4.157889e-03,1.042171e-03,-1.738348e-03,-7.629364e-04,0.313000
75%,2.287306e-01,2.317720e-01,2.069571e-01,1.657590e-01,1.281656e-01,9.993110e-02,1.272074e-01,9.657091e-02,1.002793e-01,9.059699e-02,8.833988e-02,7.148089e-02,6.778119e-02,7.575507e-02,7.118075e-02,6.354093e-02,0.794250
max,1.597730e+00,1.382802e+00,1.010250e+00,1.448007e+00,1.034062e+00,1.358963e+00,6.191600e-01,7.370014e-01,6.447869e-01,5.839577e-01,6.405349e-01,6.779764e-01,5.157227e-01,4.978796e-01,4.673081e-01,4.570815e-01,2.538000


Split Data

In [6]:
new_train_pca_16 = new_train_pca_16.fillna(0)
train = new_train_pca_16[new_test_pca_16.columns]
target = new_train_pca_16['target']

train_data, test_data, train_target, test_target = train_test_split(train, target, test_size=0.2, random_state=0)

Linear Regression

In [7]:
clf = LinearRegression()
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("LinearRegression: ", score)

LinearRegression:  0.27170117426273427


K-Neighbors Regression

In [8]:
clf = KNeighborsRegressor(n_neighbors=8)
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("KNeighborsRegression: ", score)

KNeighborsRegression:  0.2730478217182093


Random Forest Regression

In [9]:
clf = RandomForestRegressor()
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("RandomForestRegressor: ", score)

RandomForestRegressor:  0.2550231710032872


LGB Regression

In [10]:
import lightgbm as lgb
clf = lgb.LGBMRegressor(learning_rate=0.01, max_depth=-1,
                        n_estimators=5000, boosting_type='gbdt',
                        random_state=2021, objective='regression')

clf.fit(X=train_data, y=train_target, eval_metric='MSE', verbose=50)

score = mean_squared_error(test_target, clf.predict(test_data))
print("LightGBM: ", score)

LightGBM:  0.24493711836175286
